<a href="https://colab.research.google.com/github/ttogle918/Competition/blob/main/%ED%8C%94%EB%8B%B9%EB%8C%90%20%ED%99%8D%EC%88%98%20%EC%95%88%EC%A0%84%EC%9A%B4%EC%98%81%EC%97%90%20%EB%94%B0%EB%A5%B8%20%ED%95%9C%EA%B0%95%20%EC%88%98%EC%9C%84%EC%98%88%EC%B8%A1%20AI%20%EA%B2%BD%EC%A7%84%EB%8C%80%ED%9A%8C/01%20baseline_(dataset%EC%88%98%EC%A0%95).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

선 본 대회의 의도는 추론하고자 하는 기간의 10분 전에, 한강의 주요 지점(다리)에 대한 수위를 예측

    Data
    ├ Water Data
    │ ├ data_2012.csv
    │   ├ ymdhm : 년월일시분
    │   ├ swl : 팔당댐 현재수위 (단위: El.m)
    │   ├ inf : 팔당댐 유입량 (단위: m^3/s)
    │   ├ sfw : 팔당댐 저수량 (단위: 만m^3)
    │   ├ ecpc : 팔당댐 공용량 (단위: 백만m^3)
    │   ├ tototf : 총 방류량 (단위: m^3/s)
    │   ├ tide_level : 강화대교 조위 (단위: cm)
    │   ├ wl_1018662 : 청담대교 수위 (단위: cm)
    │   ├ fw_1018662 : 청담대교 유량 (단위: m^3/s)
    │   ├ wl_1018680 : 잠수교 수위 (단위: cm)
    │   ├ fw_1018680 : 잠수교 유량 (단위: m^3/s)
    │   ├ wl_1018683 : 한강대교 수위 (단위: cm)
    │   ├ fw_1018683 : 한강대교 유량 (단위: m^3/s)
    │   ├ wl_1019630 : 행주대교 수위 (단위: cm)
    │   └ fw_1019630 : 행주대교 유량 (단위: m^3/s)
    │ ├ data_2013.csv
    …
    └ └ data_2022.csv
    └ RainFall Data
    │ ├ rf_2012.csv
    │   ├ YMDHM : 년월일시분
    │   ├ rf_10184100 : 대곡교 강수량
    │   ├ rf_10184110 : 진관교 강수량
    │   └ rf_10184140 : 송정동 강수량
    │ ├ rf_2013.csv
    …
    └ └ rf_2022.csv

In [1]:
import pandas as pd
import numpy as np

from glob import glob
from tqdm import tqdm
from scipy import interpolate

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, LSTM, GRU, AveragePooling1D, GlobalAveragePooling1D

'''#GPU 자원이 부족한 경우 아래 코드를 이용하세요
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)'''

'#GPU 자원이 부족한 경우 아래 코드를 이용하세요\nfrom tensorflow.compat.v1 import ConfigProto\nfrom tensorflow.compat.v1 import InteractiveSession\n\nconfig = ConfigProto()\nconfig.gpu_options.allow_growth = True\nsession = InteractiveSession(config=config)'

In [2]:
path = '/content/drive/MyDrive/Colab Notebooks/dataset/dacon/competition_data.zip'

import zipfile
from zipfile import ZipFile

with zipfile.ZipFile(f'{path}', 'r') as obj:
  namelist = obj.namelist()
  obj.extractall('/content/competition_data')
namelist

['rf_data/',
 'rf_data/rf_2012.csv',
 'rf_data/rf_2013.csv',
 'rf_data/rf_2014.csv',
 'rf_data/rf_2015.csv',
 'rf_data/rf_2016.csv',
 'rf_data/rf_2017.csv',
 'rf_data/rf_2018.csv',
 'rf_data/rf_2019.csv',
 'rf_data/rf_2020.csv',
 'rf_data/rf_2021.csv',
 'rf_data/rf_2022.csv',
 'sample_submission.csv',
 'water_data/',
 'water_data/data_2012.csv',
 'water_data/data_2013.csv',
 'water_data/data_2014.csv',
 'water_data/data_2015.csv',
 'water_data/data_2016.csv',
 'water_data/data_2017.csv',
 'water_data/data_2018.csv',
 'water_data/data_2019.csv',
 'water_data/data_2020.csv',
 'water_data/data_2021.csv',
 'water_data/data_2022.csv']

In [89]:
w_list = sorted(glob("competition_data/water_data/*.csv"))
rf_list = sorted(glob("competition_data/rf_data/*.csv"))
w_list, rf_list

(['competition_data/water_data/data_2012.csv',
  'competition_data/water_data/data_2013.csv',
  'competition_data/water_data/data_2014.csv',
  'competition_data/water_data/data_2015.csv',
  'competition_data/water_data/data_2016.csv',
  'competition_data/water_data/data_2017.csv',
  'competition_data/water_data/data_2018.csv',
  'competition_data/water_data/data_2019.csv',
  'competition_data/water_data/data_2020.csv',
  'competition_data/water_data/data_2021.csv',
  'competition_data/water_data/data_2022.csv'],
 ['competition_data/rf_data/rf_2012.csv',
  'competition_data/rf_data/rf_2013.csv',
  'competition_data/rf_data/rf_2014.csv',
  'competition_data/rf_data/rf_2015.csv',
  'competition_data/rf_data/rf_2016.csv',
  'competition_data/rf_data/rf_2017.csv',
  'competition_data/rf_data/rf_2018.csv',
  'competition_data/rf_data/rf_2019.csv',
  'competition_data/rf_data/rf_2020.csv',
  'competition_data/rf_data/rf_2021.csv',
  'competition_data/rf_data/rf_2022.csv'])

In [90]:
df = pd.DataFrame(pd.merge(pd.read_csv(w_list[0]), pd.read_csv(rf_list[0])))
for w, rf in zip(w_list[1:], rf_list[1:]) :
  df_sample = pd.merge(pd.read_csv(w), pd.read_csv(rf))
  df = df.append(df_sample)
  print(df.shape)
df = df.reset_index(drop=True)
df = df.sort_values(by=['ymdhm'], axis=0)
df.head(3)

(52992, 18)
(79488, 18)
(105984, 18)
(132480, 18)
(158976, 18)
(185472, 18)
(211968, 18)
(238464, 18)
(264960, 18)
(276336, 18)


,ymdhm,swl,inf,sfw,ecpc,tototf,tide_level,wl_1018662,fw_1018662,wl_1018680,fw_1018680,wl_1018683,fw_1018683,wl_1019630,fw_1019630,rf_10184100,rf_10184110,rf_10184140
0,2012-05-01 00:00,24.800,555.0,219.07,24.93,555.0,445.0,310.7,469.05,300.2,0.0,290.0,729.80,275.3,540.18,0.0,0.0,0.0
1,2012-05-01 00:10,24.794,464.6,218.86,25.15,562.9,449.0,314.7,498.00,300.2,0.0,290.0,731.48,275.3,540.18,0.0,0.0,0.0
2,2012-05-01 00:20,24.789,478.1,218.69,25.31,576.4,451.0,313.7,490.68,301.2,0.0,290.0,726.42,275.3,540.18,0.0,0.0,0.0


In [91]:
na_idxes = {}
print(f'shape : {df.shape}')
print(f'null : {df.isna().sum()}')
for col in df.columns :
  na_idxes[col] = df[df[col].isna() == True].index
df = df.fillna(df.interpolate())
print(f'null : {df.isna().sum()}')

shape : (276336, 18)
null : ymdhm               0
swl               743
inf               743
sfw               743
ecpc              743
tototf            743
tide_level       4927
wl_1018662         59
fw_1018662      16380
wl_1018680         59
fw_1018680     196849
wl_1018683         59
fw_1018683       1279
wl_1019630         59
fw_1019630         59
rf_10184100         0
rf_10184110         0
rf_10184140         0
dtype: int64
null : ymdhm          0
swl            0
inf            0
sfw            0
ecpc           0
tototf         0
tide_level     0
wl_1018662     0
fw_1018662     0
wl_1018680     0
fw_1018680     0
wl_1018683     0
fw_1018683     0
wl_1019630     0
fw_1019630     0
rf_10184100    0
rf_10184110    0
rf_10184140    0
dtype: int64


In [93]:
# 단위 맞추기
df['sfw'] = df['sfw']*10000
df['ecpc'] = df['ecpc']*1000000

In [102]:
features = ['swl', 'inf', 'sfw', 'ecpc', 'tototf', 'tide_level', 'fw_1018662', 'fw_1018680', 'fw_1018683', 'fw_1019630', 'rf_10184100', 'rf_10184110', 'rf_10184140']
label = ['wl_1018662', 'wl_1018680', 'wl_1018683', 'wl_1019630']

In [ ]:
train_data = []
train_label = []

# data : ['팔당댐 현재수위', '팔당댐 유입량', '팔당댐 저수량', '팔당댐 공용량', '총 방류량', '강화대교 조위',
# '청담대교 유량',  '잠수교 유량', '한강대교 유량', '행주대교 유량']
# label : ['청담대교 수위',  '잠수교 수위', '한강대교 수위', '행주대교 수위']
for j in tqdm(range(len(df)-512)):
    train_data.append(np.array(df.loc[j:j + 511, features]).astype(float))
    train_label.append(np.array(df.loc[j + 512, label]).astype(float))

train_data = np.array(train_data)
train_label = np.array(train_label)

print(train_data.shape)
print(train_label.shape)

100%|██████████| 275824/275824 [05:05<00:00, 903.16it/s]


## 모델링

In [8]:
input_shape = (train_data[0].shape[0], train_data[0].shape[1])

model = Sequential()
model.add(GRU(256, input_shape=input_shape))
model.add(Dense(4, activation = 'relu'))

optimizer = tf.optimizers.RMSprop(0.001)

model.compile(optimizer=optimizer,loss='mse', metrics=['mae'])

In [9]:
model.fit(train_data, train_label, epochs=10, batch_size=128)

Epoch 1/10
2037/2037 [==============================] - 91s 41ms/step - loss: 23099.7812 - mae: 105.1334
Epoch 2/10
2037/2037 [==============================] - 88s 43ms/step - loss: 925.5237 - mae: 11.3946
Epoch 3/10
2037/2037 [==============================] - 89s 44ms/step - loss: 518.0557 - mae: 8.8913
Epoch 4/10
2037/2037 [==============================] - 89s 44ms/step - loss: 423.4114 - mae: 8.4397
Epoch 5/10
2037/2037 [==============================] - 89s 44ms/step - loss: 377.7960 - mae: 8.3073
Epoch 6/10
2037/2037 [==============================] - 89s 44ms/step - loss: 328.2842 - mae: 7.9995
Epoch 7/10
2037/2037 [==============================] - 89s 44ms/step - loss: 297.0500 - mae: 7.9390
Epoch 8/10
2037/2037 [==============================] - 89s 44ms/step - loss: 294.8503 - mae: 8.0706
Epoch 9/10
2037/2037 [==============================] - 89s 44ms/step - loss: 318.0398 - mae: 8.4751
Epoch 10/10
2037/2037 [==============================] - 89s 44ms/step - loss: 370.021

In [10]:
test_data = []
test_label = []

tmp = pd.read_csv(w_list[-1])
tmp = tmp.replace(" ", np.nan)
# 이전값을 사용
tmp = tmp.fillna(method = 'pad')
tmp = tmp.fillna(0)
    
#tmp.loc[:, ["wl_1018662", "wl_1018680", "wl_1018683", "wl_1019630"]] = tmp.loc[:, ["wl_1018662", "wl_1018680", "wl_1018683", "wl_1019630"]]*100
    
for j in tqdm(range(4032, len(tmp)-432)):
    test_data.append(np.array(tmp.loc[j:j + 431, ["swl", "inf", "sfw", "ecpc",
                                                    "tototf", "tide_level",
                                                    "fw_1018662", "fw_1018680",
                                                    "fw_1018683", "fw_1019630"]]).astype(float))
        
    test_label.append(np.array(tmp.loc[j + 432, ["wl_1018662", "wl_1018680",
                                                    "wl_1018683", "wl_1019630"]]).astype(float))

100%|██████████| 6912/6912 [00:07<00:00, 898.91it/s]


In [11]:

test_data = np.array(test_data)
test_label = np.array(test_label)

print(test_data.shape)
print(test_label.shape)
test_label

(6912, 432, 10)
(6912, 4)


array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [12]:
pred = model.predict(test_data)

In [13]:
pred = pd.DataFrame(pred)

In [14]:
sample_submission = pd.read_csv("competition_data/sample_submission.csv")

sample_submission["wl_1018662"] = pred[0]
sample_submission["wl_1018680"] = pred[1]
sample_submission["wl_1018683"] = pred[2]
sample_submission["wl_1019630"] = pred[3]

In [15]:
sample_submission.to_csv("baseline.csv", index = False)

In [16]:
pred.head()

,0,1,2,3
0,282.492126,272.556580,268.848572,261.440430
1,282.494232,272.558624,268.850708,261.442474
2,282.636902,272.694763,268.994049,261.580231
3,285.243469,275.181915,271.612823,264.096863
4,282.545471,272.607544,268.902222,261.491943


In [17]:
df_test_label = pd.DataFrame({0:test_label[:, 0], 1:test_label[:, 0], 2:test_label[:, 0], 3:test_label[:, 0]})
df_test_label

,0,1,2,3
0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0
...,...,...,...,...
6907,0.0,0.0,0.0,0.0
6908,0.0,0.0,0.0,0.0
6909,0.0,0.0,0.0,0.0
6910,0.0,0.0,0.0,0.0


In [18]:
pred.shape

(6912, 4)

In [19]:
df_test_label.max()

0    0.0
1    0.0
2    0.0
3    0.0
dtype: float64